## Imports

In [1]:
from agents import Agent, Runner, function_tool, trace
from dotenv import load_dotenv
import asyncio
import os
from typing import Dict
from openai.types.responses import ResponseTextDeltaEvent
from IPython.display import Markdown, display
import requests

In [2]:
load_dotenv(override=True)

True

In [3]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## 1. Create Agentic Workflow

We will be copying everything from exercise 2

In [6]:
# part 1: principle agent.
# instructions
math_instructions = "You are a 5th grade math teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Math**. Help the students the best you can."

science_instructions = "You are a 5th grade science teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Science**. Help the students the best you can."

humanities_instructions = "You are a 5th grade humanities teacher at a new school called: Memo Elementary School.\
    You are a generic 5th grade teacher but specialize in **Humanities**. Help the students the best you can."

teacher_picker_instructions = """
You are a Principle that oversees an after-school tutoring program for 5th graders.

## Inputs
- Background information on student wanting a tutor
- Introductions from a few teachers that are willing to offer assistance 

## Output
Create a 2-3 sentence summary of what teacher you have chosen for the student and why
"""

# agents.
math_agent = Agent (
    name="math_agent",
    instructions=math_instructions,
    model="gpt-4o-mini"
)

science_agent = Agent (
    name="science_agent",
    instructions=science_instructions,
    model="gpt-4o-mini"
)

humanities_agent = Agent (
    name="humanities_agent",
    instructions=humanities_instructions,
    model="gpt-4o-mini"
)

teacher_picker_agent = Agent(
    name="teacher_picker_agent",
    instructions=teacher_picker_instructions,
    model="gpt-4o-mini"
)

# tools.
math_tool = math_agent.as_tool(
    tool_name="math_teacher", 
    tool_description="General 5th grade teacher that specializes in Math"
)

science_tool = math_agent.as_tool(
    tool_name="science_teacher", 
    tool_description="General 5th grade teacher that specializes in Science"
)

humanities_tool = math_agent.as_tool(
    tool_name="humanities_teacher", 
    tool_description="General 5th grade teacher that specializes in Humanities"
)

teacher_picker_tool = teacher_picker_agent.as_tool(
    tool_name="teacher_picker",
    tool_description="Choose a teacher to tutor a student"
)

teacher_tools = [math_tool, science_tool, humanities_tool, teacher_picker_tool]


# part 2: messenger agent (handoff agent).
# instructions.
subject_instructions = "You craft a short and brief subject line for a given message"
body_instructions = "You craft a short and brief message body based on information recieved"

# agents.
subject_agent = Agent(name="Subject Agent", instructions=subject_instructions, model="gpt-4o-mini")
body_agent = Agent(name="Body Agent", instructions=body_instructions, model="gpt-4o-mini")

# tools.
subject_tool = subject_agent.as_tool(
    tool_name="subject_tool",
    tool_description = "General email subject creator"
)

body_tool = body_agent.as_tool(
    tool_name="body_tool",
    tool_description = "General email body creator"
)

@function_tool
def send_message(subject: str, body: str) -> str:
    message = f"""
    {subject}
    
    {body}
    """
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

messenger_tools = [subject_tool,  body_tool, send_message]

# define handoff agent.
messenger_instructions = """You are a messager that is supposed to use a teacher's introduction and create/send a message to a student about offering after-school assistance \

Follow these steps exactly:    
1. Use the subject writer tool to generate a subject line for the message.
2. Use the body writer tool to generate a message body.
3. Use the send message tool to pass to send a message to the student. Pass the outputs from steps 1 and 2.
4. Return the subject and body of the message you just sent.
"""

messenger_agent = Agent(
    name="messenger_agent", 
    instructions=messenger_instructions, 
    model="gpt-4o-mini",
    tools=messenger_tools,
    handoff_description="Messenge student who their teacher is for after-school support"
)

handoffs = [messenger_agent]

# define main principle agent.
instructions = """
You are a 5th grade Principle that delegates each student to the **best** teacher for an after-school tutoring program. 
You recieve background information on the student. 
Follow these steps exactly: 
1. Use the teacher tools to recieve background information on each of the teachers. 
2. Use the teacher generator tool to assign a teacher for the student. 
3. Pass ONLY the output from the teacher generator tool to the 'Messenger Agent' agent. 
4. Output exactly what the Messenger Agent returned, word for word.
"""

princple_agent = Agent(
    name="princple_agent",
    instructions=instructions,
    tools=teacher_tools,
    handoffs=handoffs,
    model="gpt-4o-mini"
)

with trace("Teacher Picker using Push Tool"):
    result = Runner.run_streamed(princple_agent, "I am a student struggling with Math")
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)

The message has been successfully sent. Here are the details:

**Subject:** After-School Math Assistance

**Body:** 
Dear Student,

I hope you're doing well! I’m offering after-school math assistance and would love to help you strengthen your skills.

Please let me know which days work for you, and we can arrange a time to meet.

Looking forward to hearing from you!

Best,  
Ms. Anderson

## Adding Guardrails

As you can see, there is a Ms. Anderson. That is good, otherwise would be kinda sketchy for a student to recieve this message. let's add an agent that will act as our guardrail

in this case, we want it to trigger if it DOES NOT contain a name

In [13]:
from agents import GuardrailFunctionOutput, input_guardrail
from pydantic import BaseModel

In [18]:
class NameCheckOutput(BaseModel):
    contains_name_not: str
    name: str

In [17]:
guardrail_agent = Agent(
    name="Guardrail Agent",
    instructions="Check if the output body message does NOT contain the teacher's name",
    output_type=NameCheckOutput,     # output_type autofills this class based on output/instructions
    model="gpt-4o-mini"
)

In [21]:
@input_guardrail
async def guardrail_against_name(ctx, agent, message):
    result = await Runner.run(guardrail_agent, message, context=ctx)
    contains_name_not = result.final_output.contains_name_not
    return GuardrailFunctionOutput(output_info={"output": result.final_output}, tripwire_triggered=contains_name_not)

Let's redefine our Principle to include this guardrail

In [25]:
princple_agent_guardrail = Agent(
    name="Principle Agent Guardrail",
    instructions=instructions,
    tools=teacher_tools,
    handoffs=handoffs,
    model="gpt-4o-mini",
    input_guardrails=[guardrail_against_name]
)

In [26]:
with trace("test"):
    result = await Runner.run(princple_agent_guardrail, "I struggle with science!")

InputGuardrailTripwireTriggered: Guardrail InputGuardrail triggered tripwire